In [1]:
import glob

import numpy as np
import pandas as pd

from grafting_classifier import GraftingClassifier
from sklearn.linear_model import SGDClassifier
from ogfs_classifier import OGFSClassifier
from dpp_classifier import DPPClassifier
from dpp_classifier_mitra import DPPClassifier as DPPClassifier2

from sklearn.metrics import log_loss, accuracy_score

#import dask.dataframe as dd
#import dask.array as da

In [2]:
class_train = glob.glob("microarray/*_train.csv")
print(class_train)

['microarray\\colon_train.csv', 'microarray\\leukemia_train.csv', 'microarray\\lung_cancer_train.csv', 'microarray\\prostate_train.csv']


In [3]:
def train_label(fname):
    targetname = fname.replace(".csv", ".labels")
    return pd.read_csv(targetname)

In [4]:
def get_performance(mod, fpath, base=False):
    train1 = pd.read_csv(fpath).fillna(0)
    y = np.array(train_label(fpath)).flatten()
    
    # simulate streaming...
    # try splitting into groups of ~10,
    # if there is no splits, try ~5.
    train1_cols = np.array_split(range(train1.shape[1]), int(train1.shape[1]/10.0) + 1)
    if len(train1_cols) == 1:
        train1_cols = np.array_split(range(train1.shape[1]), int(train1.shape[1]/5.0) + 1)
    all_cols = []

    #mod = GraftingClassifier(max_iter=5)
    if base:
        mod.fit(train1, y)
        results = {'accuracy': accuracy_score(y, mod.predict(train1)), 
               'logloss': log_loss(y, mod.predict_proba(train1)), 
               'feat_dim': mod.coef_.flatten().shape}
        return results

    for idx, collist in enumerate(train1_cols):
        if idx == 0:
            column_list = list(np.array(list(train1.columns))[collist])
            mod.fit(train1[column_list], y)
            all_cols.extend(list(collist))
        else:
            all_cols.extend(list(collist))
            column_list = list(np.array(list(train1.columns))[all_cols])
            mod.partial_fit(train1[column_list], y)
    
    results = {'accuracy': accuracy_score(y, mod.predict(train1)), 
               'logloss': log_loss(y, mod.predict_proba(train1)), 
               'feat_dim': mod.coef_.flatten().shape}
    return results

In [5]:
models = [
    ('Grafting', GraftingClassifier(max_iter=5, random_state=42)), 
    ('DPP', DPPClassifier(max_iter=5, random_state=42)), 
    ('DPP2', DPPClassifier2(max_iter=5, random_state=42)),
    #('OGFS', OGFSClassifier(max_iter=5, random_state=42)),
    ('Base', SGDClassifier(loss='log', max_iter=5, random_state=42))
]

In [6]:
ex_dat = class_train[0]
for nm, mod in models:
    if nm != 'Base':
        print(nm, get_performance(mod, ex_dat))
    else:
        print(nm, get_performance(mod, ex_dat, base=True))

C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp.py:233: RuntimeWarning: divide by zero encountered in true_divide
  V_norm = V[choose_item, :]/Vj[choose_item]
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp.py:215: RuntimeWarning: invalid value encountered in less
  choose_item = np.random.choice(range(len(P_list)), 1, p=P_norm)[0]


Grafting {'accuracy': 1.0, 'logloss': 9.9920072216264148e-16, 'feat_dim': (490,)}


C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2397: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2422: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


DPP {'accuracy': 0.80645161290322576, 'logloss': 6.6849244635311003, 'feat_dim': (84,)}
DPP2 {'accuracy': 0.66129032258064513, 'logloss': 11.698617811179425, 'feat_dim': (81,)}
Base {'accuracy': 0.66129032258064513, 'logloss': 11.698617811179425, 'feat_dim': (2000,)}


In [7]:
ex_dat = class_train[1]
for nm, mod in models:
    if nm != 'Base':
        print(nm, get_performance(mod, ex_dat))
    else:
        print(nm, get_performance(mod, ex_dat, base=True))

C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


Grafting {'accuracy': 1.0, 'logloss': 9.9920072216264148e-16, 'feat_dim': (866,)}


C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp.py:233: RuntimeWarning: divide by zero encountered in true_divide
  V_norm = V[choose_item, :]/Vj[choose_item]
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp.py:215: RuntimeWarning: invalid value encountered in less
  choose_item = np.random.choice(range(len(P_list)), 1, p=P_norm)[0]
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2397: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2422: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\chapm\Anaconda3\envs\skre

DPP {'accuracy': 0.97222222222222221, 'logloss': 0.95941045541418646, 'feat_dim': (109,)}
DPP2 {'accuracy': 0.40277777777777779, 'logloss': 20.627324791404991, 'feat_dim': (75,)}
Base {'accuracy': 1.0, 'logloss': 9.9920072216264148e-16, 'feat_dim': (7129,)}


In [ ]:
ex_dat = class_train[2]
for nm, mod in models:
    if nm != 'Base':
        print(nm, get_performance(mod, ex_dat))
    else:
        print(nm, get_performance(mod, ex_dat, base=True))

C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


Grafting {'accuracy': 1.0, 'logloss': 9.9920072216264128e-16, 'feat_dim': (140,)}


C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp.py:233: RuntimeWarning: divide by zero encountered in true_divide
  V_norm = V[choose_item, :]/Vj[choose_item]
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp.py:215: RuntimeWarning: invalid value encountered in less
  choose_item = np.random.choice(range(len(P_list)), 1, p=P_norm)[0]
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2397: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2422: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\chapm\Anaconda3\envs\skre

DPP {'accuracy': 0.93922651933701662, 'logloss': 2.0990416593592136, 'feat_dim': (48,)}
DPP2 {'accuracy': 0.94475138121546964, 'logloss': 1.9082196903265574, 'feat_dim': (210,)}
Base {'accuracy': 1.0, 'logloss': 9.9920072216264128e-16, 'feat_dim': (12533,)}


In [ ]:
ex_dat = class_train[3]
for nm, mod in models:
    if nm != 'Base':
        print(nm, get_performance(mod, ex_dat))
    else:
        print(nm, get_performance(mod, ex_dat, base=True))

C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


Grafting {'accuracy': 1.0, 'logloss': 9.9920072216264128e-16, 'feat_dim': (1432,)}


C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp.py:233: RuntimeWarning: divide by zero encountered in true_divide
  V_norm = V[choose_item, :]/Vj[choose_item]
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp.py:215: RuntimeWarning: invalid value encountered in less
  choose_item = np.random.choice(range(len(P_list)), 1, p=P_norm)[0]
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2397: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2422: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\chapm\Anaconda3\envs\skre

DPP {'accuracy': 0.52941176470588236, 'logloss': 16.253541832899145, 'feat_dim': (69,)}


C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp.py:233: RuntimeWarning: overflow encountered in true_divide
  V_norm = V[choose_item, :]/Vj[choose_item]
